# The Stepsize of DSEA+

DSEA+ extends the original DSEA with an adaptively chosen stepsize between iterations. This tutorial assumes you already know the other notebook at `example/getting-started.ipynb`.

In [1]:
using CherenkovDeconvolution
using ScikitLearn, MLDataUtils, Random
using Discretizers: encode, CategoricalDiscretizer

# load the example data, encode labels with integers
X, y_labels, _ = load_iris()
y = encode(CategoricalDiscretizer(y_labels), y_labels)

# split the data into training and observed data sets
Random.seed!(42) # make split reproducible
(X_train, y_train), (X_data, y_data) = splitobs(shuffleobs((X', y), obsdim = 1), obsdim = 1)

# discretize the feature space
td = DeconvLearn.TreeDiscretizer(X_train, y_train, 3) # obtain up to 3 clusters
x_train = encode(td, X_train)
x_data  = encode(td, X_data)

# also prepare the classifier for DSEA
@sk_import naive_bayes : GaussianNB
tp_function = DeconvLearn.train_and_predict_proba(GaussianNB());

## Adaptive Step Size

The adaptive step size is specified through the `alpha` argument of DSEA. This argument expects a `Function` object, for which CherenkovDeconvolution.jl provides several factory methods.

The most important factory uses the objective function of the regularized unfolding (RUN) to determine the step size adaptively. We further specify `epsilon`, the minimum Chi square distance between iterations. Convergence is assumed if the distance drops below this threshold.

In this example, convergence is assumed immediately.

In [2]:
alpha_function = alpha_adaptive_run(x_data, x_train, convert(Vector{Int64}, y_train)) # returns a Function object
f_dsea = dsea(X_data, X_train, y_train, tp_function, K=100, epsilon=1e-6, alpha=alpha_function)

┌ Info: DSEA iteration 1/100 uses alpha = 2.828411083473901e-13 (chi2s = 2.237160223400213e-28)
└ @ CherenkovDeconvolution /home/bunse/.julia/dev/CherenkovDeconvolution/src/methods/dsea.jl:154
┌ Info: DSEA convergence assumed from chi2s = 2.237160223400213e-28 < epsilon = 1.0e-6
└ @ CherenkovDeconvolution /home/bunse/.julia/dev/CherenkovDeconvolution/src/methods/dsea.jl:159


3-element Array{Float64,1}:
 0.3333333333333333
 0.3333333333333394
 0.3333333333333272

## Other Step Sizes

Two decaying step sizes can be obtained with `alpha_decay_exp` and `alpha_decay_mul`. For constant step sizes, a `Float64` can directly be given as the `alpha` parameter.

You can further define custom `alpha` functions. In general, such a function has to have the following signature:

    (k::Int, pk::Vector{Float64}, f::Vector{Float64}) -> Float64

The first parameter in this signature specifies the current iteration number. `pk` defines the step, i.e. the difference between the next and current estimate. Finally, `f` gives the current estimate before the step `pk` is taken.

## Further Documentation

In [3]:
?alpha_adaptive_run

search: alpha_adaptive_run



```
alpha_adaptive_run(x_data, x_train, y_train[, tau=0; bins_y, bins_x, warn=false])
```

Return a `Function` object with the signature required by the `alpha` parameter in `dsea`. This object adapts the DSEA step size to the current estimate by maximizing the likelihood of the next estimate in the search direction of the current iteration.


In [4]:
?alpha_decay_exp

search: alpha_decay_exp alpha_decay_mul



```
alpha_decay_exp(eta::Float64, a_1::Float64=1.0)
```

Return a `Function` object with the signature required by the `alpha` parameter in `dsea`. This object reduces the `a_1` stepsize taken in iteration 1 by `eta` in each subsequent iteration:

```
alpha = a_1 * eta^(k-1).
```


In [5]:
?alpha_decay_mul

search: alpha_decay_mul alpha_decay_exp



```
alpha_decay_mul(eta::Float64, a_1::Float64=1.0)
```

Return a `Function` object with the signature required by the `alpha` parameter in `dsea`. This object reduces the `a_1` stepsize taken in iteration 1 by `eta` in each subsequent iteration:

```
alpha = a_1 * k ^ (eta-1)
```

For example, eta=.5 yields alpha = 1/sqrt(k).
